In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, array_contains
import pyspark
import re
from pyspark.sql.functions import collect_set
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import concat_ws
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = 'spark-shell --packages com.databricks:spark-xml_2.11:0.12.0'

spark = SparkSession.builder.master('local[*]').appName('VINF').getOrCreate()

22/12/03 17:52:22 WARN Utils: Your hostname, Adrians-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.183 instead (on interface en0)
22/12/03 17:52:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/12/03 17:52:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Exception in thread "main" org.apache.spark.SparkException: Failed to get main class in JAR with error 'File spark-shell does not exist'.  Please specify one with --class.
	at org.apache.spark.deploy.SparkSubmit.error(SparkSubmit.scala:975)
	at org.apache.spark.deploy.SparkSubmit.prepareSubmitEnvironment(SparkSubmit.scala:486)
	at org.apache.spark.deploy.SparkSubmit.org$apache$spark$deploy$SparkSubmit$$runMain(SparkSubmit.scala:901)
	at org.apache.spark.deploy.SparkSubmit.doRunMain$1(SparkSubmit.scala:180)
	at org.apache.spark.deploy.SparkSubmit.submit(SparkSubmit.scala:203)
	at org.apache.spark.deploy.SparkSubmit.doSubmit(SparkSubmit.scala:90)
	at org.apache.spark.deploy.SparkSubmit$$anon$2.doSubmit(SparkSubmit.scala:1046)
	at org.apache.spark.deploy.SparkSubmit$.main(SparkSubmit.scala:1055)
	at org.apache.spark.deploy.SparkSubmit.main(SparkSubmit.scala)


RuntimeError: Java gateway process exited before sending its port number

In [ ]:
df = spark.read.format("com.databricks.spark.xml").load('freebase-head-10000000')

In [89]:
book_regex = "^book\.book$"

book_editions_regex = "^book\.book\.editions$"
book_edition_regex = "book\.book_edition"
book_author_regex = "book\.written_work\.author"
book_author = "book\.author"
book_genre_regex = "book\.book\.genre"
name_regex = "type\.object\.name"
year_of_publication = "book.written_work.date_of_first_publication"
description_regex = "common.topic.description"
alias_regex = "common.topic.alias"
isbn_regex = "media_common.cataloged_instance.isbn13"
publication_date_regex = "book.book_edition.publication_date"
en_string = "@en$"

remove_pattern = '(http\:\/\/rdf.freebase.com\/ns\/)|(\^\^.*)|(\@.*\.)|(\<)|(\>)|(\")|(\t\.)'
file_path = "./freebase-head-10000000"

In [90]:
rdd = spark.sparkContext.textFile(file_path)

In [91]:
all_rdd = rdd.map(lambda x: re.sub('\t\.', '', x)) \
    .map(lambda x: re.sub(remove_pattern, '', x)) \
    .map(lambda x: x.split('\t'))
#     .map(lambda x: [x[0], x[1], re.sub("\\\"((\d+-?)+)\\\".+XMLSchema.+", r"\1", x[2])])

In [92]:
schema = StructType([StructField('subject', StringType(), False),
                StructField('predicate', StringType(), False),
                StructField('object', StringType(), False)])

## Book editions

In [93]:
book_editions_rdd = all_rdd.filter(lambda x: re.search(book_edition_regex, x[2]))

In [94]:
book_edition_columns = all_rdd.filter(lambda x: (re.search(name_regex, x[1]) and re.search(en_string, x[2])) or re.search(isbn_regex, x[1]) or re.search(publication_date_regex, x[1]))

In [95]:
book_editions_id_df = spark.createDataFrame(book_editions_rdd, schema = schema).select('subject').distinct()

In [96]:
book_editions_id_df = book_editions_id_df.dropDuplicates()

In [97]:
book_edition_columns_df = spark.createDataFrame(book_edition_columns, schema = schema)

In [98]:
book_editions_id_df.createOrReplaceTempView("book_editions_id")

In [99]:
book_edition_columns_df.createOrReplaceTempView("book_editions_columns")

In [100]:
books_editions_dataframe = spark.sql("""
    SELECT book_editions_id.subject, book_editions_columns.predicate, book_editions_columns.object
    from book_editions_id
    LEFT JOIN book_editions_columns on book_editions_id.subject == book_editions_columns.subject
    ORDER BY book_editions_id.subject
""")

In [101]:
books_editions_dataframe.show(truncate=False)

+----------+--------------------------------------+-----------------------------------------------------------------------------------------------+
|subject   |predicate                             |object                                                                                         |
+----------+--------------------------------------+-----------------------------------------------------------------------------------------------+
|m.0101nvp6|book.book_edition.publication_date    |2014-03-18                                                                                     |
|m.0101nvp6|media_common.cataloged_instance.isbn13|9781455599509                                                                                  |
|m.0101nvp6|type.object.name                      |You Should Have Known@en                                                                       |
|m.010pjrs8|type.object.name                      |Nathan Hale's Hazardous Tales: Treaties, Trenches, Mud, and B

In [103]:
book_editions_grouped = books_editions_dataframe.groupBy(['subject', 'predicate']).agg(collect_list('object').alias('object'))

book_editions_rdd = book_editions_grouped.rdd
book_editions_rdd = book_editions_rdd.map(lambda x: (x[0], (x[1], x[2])))
book_editions_rdd = book_editions_rdd.groupByKey().map(lambda x: (x[0], { '\''+str(name)+'\'' + ':' + str(value) for name, value in list(x[1])}))

book_editions_dictionary = book_editions_rdd.collect()

schema_id = StructType([ \
    StructField("id",StringType(),False), \
    StructField("properties",StringType(),True)
  ])

book_editions_df = spark.createDataFrame(data=book_editions_dictionary, schema = schema_id)
# book_editions_df = spark.createDataFrame(data=book_editions_dictionary, schema = ["id","properties"])


# book_editions_df.printSchema()
# book_editions_df.show(truncate=False)
book_editions_df.createOrReplaceTempView("book_editions_json")

## Authors and genres table

In [104]:
author_columns = all_rdd.filter(lambda x: (re.search(name_regex, x[1]) and re.search(en_string, x[2])))

In [105]:
author_genres_ids_rdd = all_rdd.filter(lambda x: re.search(book_author, x[2]) or re.search(book_genre_regex, x[2]))

In [106]:
author_genres_book_df = spark.createDataFrame(author_genres_ids_rdd, schema = schema).select('subject').distinct()

author_df = spark.createDataFrame(author_columns, schema = schema)

author_df = author_df.dropDuplicates()

In [107]:
author_df = author_df.dropDuplicates()

In [108]:
author_df.createOrReplaceTempView("AUTHOR_COLUMNS")

In [109]:
author_genres_book_df.createOrReplaceTempView("AUTHOR_DATA")

In [110]:
author_dataframe = spark.sql("""
    SELECT AUTHOR_DATA.subject, AUTHOR_COLUMNS.predicate, AUTHOR_COLUMNS.object
    from AUTHOR_DATA
    LEFT JOIN AUTHOR_COLUMNS on AUTHOR_DATA.subject == AUTHOR_COLUMNS.subject
    ORDER BY AUTHOR_DATA.subject
""")

In [111]:
authors_grouped = author_dataframe.groupBy(['subject']).agg(collect_list('object').alias('object'))

# authors_grouped.show(1, False, True)

In [112]:
authors_grouped.createOrReplaceTempView("AUTHOR_GENRE_NAMES")

## Book table

In [113]:
book_ids_rdd = all_rdd.filter(lambda x: re.search(book_regex, x[2]))

In [114]:
book_columns = all_rdd.filter(lambda x: re.search(book_editions_regex, x[1]) or re.search(book_author_regex, x[1]) or re.search(book_genre_regex, x[1]) or (re.search(name_regex, x[1]) and re.search(en_string, x[2])) or re.search(year_of_publication, x[1]) or (re.search(description_regex, x[1]) and re.search(en_string, x[2])))

In [115]:
book_id_df = spark.createDataFrame(book_ids_rdd, schema = schema).select('subject').distinct()

In [116]:
book_df = spark.createDataFrame(book_columns, schema = schema)

In [117]:
book_df = book_df.dropDuplicates()

In [118]:
book_df.createOrReplaceTempView("BOOK_COLUMNS")

In [119]:
book_id_df.createOrReplaceTempView("BOOK_ID")

In [120]:
books_dataframe = spark.sql("""
    SELECT BOOK_ID.subject, BOOK_COLUMNS.predicate, BOOK_COLUMNS.object
    from BOOK_ID
    JOIN BOOK_COLUMNS on BOOK_ID.subject == BOOK_COLUMNS.subject
    ORDER BY BOOK_ID.subject
""")

In [121]:
books_dataframe.createOrReplaceTempView("BOOK_DATA")

In [122]:
books_grouped = books_dataframe.groupBy(['subject', 'predicate']).agg(collect_list('object').alias('object'))

# books_grouped.show(1, False, True)

In [123]:
# grouped_rdd = books_grouped.rdd

In [124]:
# grouped_rdd = books_grouped.rdd
# grouped_rdd2 = grouped_rdd.map(lambda x: (x[0], (x[1], x[2])))
# grouped_rdd2.collect()

In [125]:
# grouped_rdd3 = grouped_rdd2.groupByKey().map(lambda x: (x[0], { name:value for name, value in list(x[1])}))
# grouped_rdd3.collect()

In [126]:
# dictionary_data = grouped_rdd3.collect()

## Spojenie zaznamov



In [127]:
books_dataframe = spark.sql("""
    SELECT BOOK_DATA.subject, BOOK_DATA.predicate, 
    CASE 
        when AUTHOR_GENRE_NAMES.subject is not null then (cast(AUTHOR_GENRE_NAMES.object as STRING))
        when book_editions_json.id is not null then book_editions_json.properties
        when AUTHOR_GENRE_NAMES.subject is null and book_editions_json.id is null then BOOK_DATA.object
    end as object
    from BOOK_DATA
    LEFT JOIN AUTHOR_GENRE_NAMES on BOOK_DATA.object == AUTHOR_GENRE_NAMES.subject
    LEFT JOIN book_editions_json on BOOK_DATA.object == book_editions_json.id
    ORDER BY BOOK_DATA.subject
""")

In [128]:
books_grouped = books_dataframe.groupBy(['subject', 'predicate']).agg(collect_list('object').alias('object'))
grouped_rdd = books_grouped.rdd
grouped_rdd = grouped_rdd.map(lambda x: (x[0], (x[1], x[2])))
grouped_rdd = grouped_rdd.groupByKey().map(lambda x: {x[0]: { name:value for name, value in list(x[1])}})

In [129]:
# grouped_rdd.collect()

In [130]:
grouped_rdd.coalesce(1, True).saveAsTextFile('outputs10m')

# Koniec prvej casti